In [1]:

from gensim.models import FastText,Word2Vec
#import comman_function_analysis as cf
#import commonFunctions as cff
import os
os.environ["MKL_NUM_THREADS"] = "2" 
os.environ["NUMEXPR_NUM_THREADS"] = "2" 
os.environ["OMP_NUM_THREADS"] = "2" 
import pickle
import numpy as np
import scipy
from scipy import stats
from collections import OrderedDict

import pickle
with open('concept_withName_dict.bin','rb') as fin:
    concept_dict = pickle.load(fin)
    
def getAllfiles(path, endStr='', startStr=''):
    """
    :param path: URL of the folder
    :startStr: starting text if specific named files are needed
    :return: a list of all files inside a folder

    """
    if endStr!= '' and startStr!='':

        return [ y   for x in (os.walk(path)) for y in x[2] if y.endswith(endStr) and y.startswith(startStr)  ]
    else:
        return [y for x in (os.walk(path)) for y in x[2] ]
def cosine_similarity(vec1,vec2):
    
    """
    returns the normalized cosine similarity between two vector, similar to gensim consine similarity
    """
    return np.dot(vec1, vec2)/(np.linalg.norm(vec1)* np.linalg.norm(vec2))




In [19]:
import warnings
warnings.filterwarnings('ignore')

modelList = getAllfiles('/home/gvashisth/experiments/thesis/scripts/SGNS_trained_model/new_widir/','.bin','wi_')

model_dict = { file : 
              FastText.load_fasttext_format('/home/gvashisth/experiments/thesis/scripts/SGNS_trained_model/new_widir/'+file) 
              for file in modelList}

In [2]:

modelList = getAllfiles('/home/gvashisth/experiments/thesis/scripts/SGNS_trained_model/10_WC/','.bin','wi_')

model_dict_10 = { file : 
              FastText.load_fasttext_format('/home/gvashisth/experiments/thesis/scripts/SGNS_trained_model/10_WC/'+file) 
              for file in modelList}
model_dict_10

{'wi_1809_1970_fastext_10_300_5.bin': <gensim.models.fasttext.FastText at 0x7f9679fd3828>,
 'wi_1971_1980_fastext_10_300_5.bin': <gensim.models.fasttext.FastText at 0x7f9678458ac8>,
 'wi_1981_1990_fastext_10_300_5.bin': <gensim.models.fasttext.FastText at 0x7f96f8057e10>,
 'wi_1991_2000_fastext_10_300_5.bin': <gensim.models.fasttext.FastText at 0x7f9645a97c50>,
 'wi_2001_2010_fastext_10_300_5.bin': <gensim.models.fasttext.FastText at 0x7f9675593908>,
 'wi_2011_2015_fastext_10_300_5.bin': <gensim.models.fasttext.FastText at 0x7f969a643630>}

In [3]:
time_frame = ['1809_1970','1971_1980','1981_1990','1991_2000','2001_2010','2011_2015'] #complete

In [4]:
from changepoint.mean_shift_model import MeanShiftModel

In [32]:
def get_changpoint_score(model_dict,time_frame,disease,drug):
    
    score_dict,score_list = {},[]
    for time in time_frame:
        k_sgns='wi_'+time+'_fastext_10_300_5.bin' 
        model=model_dict[k_sgns]
        if disease in model.wv.vocab.keys() and drug in  model.wv.vocab.keys() :
            score_dict[time]= model.wv.similarity(disease,drug) 
            score_list.append(model.wv.similarity(disease,drug))
        else:
            score_dict[time] = 0
            score_list.append(0)
    #get the normalised score before modelshift
    _mean = np.mean(score_list)
    #_var = np.var(score_list)
    _sd = np.std(score_list)
    normalised_score = [(score-_mean)/_sd for score in score_list]
    model_change = MeanShiftModel()
    stats_ts, pvals, nums = model_change.detect_mean_shift(normalised_score, B=100)
    
    print("{}_{}|{}".format(drug,concept_dict [drug][0] if drug in concept_dict.keys() else "","|".join(map(str,score_list))))
    print("p_value||{}".format("|".join(map(str,pvals))))
    

In [29]:
def get_score_sgns_pmi_disease_drug(drug,disease,modeldict,model_start,wc):
    sgns_score, meta_info = [],[]
    for time in time_frame:
        k_sgns=model_start+time+'_fastext_'+wc+'_300_5.bin'
        if disease in modeldict[k_sgns].wv.vocab.keys() and drug in  modeldict[k_sgns].wv.vocab.keys() :
            sgns_score.append(modeldict[k_sgns].wv.similarity(disease,drug))
        else:
            sgns_score.append('0')
    print ("{}_{}|{}".format(drug, concept_dict [drug][0] if drug in concept_dict.keys() else "","|".join(map(str,sgns_score)))) 

In [71]:
surgery_list = []
with open('medication_dir/surgery','r') as fout:
    for line in fout:
        surgery_list.append(line.strip('\n'))
        


In [81]:
dopamine_list = []
with open('medication_dir/dopamine','r') as fout:
    for line in fout:
        dopamine_list.append(line.strip('\n'))
        


In [72]:
len(surgery_list)

5219

In [92]:
#prolactinoma
#'C0038895','C0543467','C1457907',"C0038894","C1274039",'C1547138','C1705196','C4084778',
for disease in ['C0344452']:
    print("{}_{}|{}".format(disease,concept_dict [disease][0] if disease in concept_dict.keys() else "","|".join(time_frame))) 
    for drug in surgery_list+dopamine_list+["C2985562","C0013030","C0201989","C0107994","C0178601",'C2757007','C0025641','C0031008','C0546852','C0700481','C0256089','C0006230'] :
        get_score_sgns_pmi_disease_drug(drug,disease,model_dict_10,model_start="wi_",wc='10')


for disease in ["C0344452",]: #,'C0344453','C0032000','C0033375','C0346302','C1306214'
    print("{}|{}".format(disease,"|".join(time_frame)))
    #print("{}_{}|{}".format(disease,concept_dict [disease][0] if disease in concept_dict.keys() else "","|".join(time_frame))) 
    for drug in surgery_list+dopamine_list+["C2985562","C0013030","C0201989","C0107994","C0178601",'C2757007','C0025641','C0031008','C0546852','C0700481','C0256089','C0006230']:
        get_changpoint_score(model_dict_10,time_frame,disease,drug)

C0344452_prolactinoma microadenoma |1809_1970|1971_1980|1981_1990|1991_2000|2001_2010|2011_2015
C0001727_syndrome; gastrojejunal loop obstruction |0|0.349616|0.238415|0.217866|0.332173|0.390126
C0002388_surgery dentoalveolar structures alveolectomy |0|0.466809|0.291546|0.221219|0.147882|0.257383
C0002393_surgery alveoloplasty |0|0.457503|0.300779|0|0.198353|0.241702
C0002428_surgery ambulatory |0|0.334022|0.162002|0.180787|0.169029|0.171702
C0005427_surgery of biliary tract (treatment) |0|0.420578|0.182346|0.200665|0.223993|0.310608
C0005976_orthopedic surgery bone grafting |0|0.313282|0.260025|0.22982|0.215451|0.232032
C0007389_cataracts surgery |0|0.340836|0.15722|0.251379|0.178883|0.223412
C0009369_Surgery.colorectal |0|0.488755|0.143413|0.221266|0.216386|0.211687
C0010055_coronary bypass surgery |0|0.326732|0.244826|0.273672|0.229301|0.269831
C0010408_cryotherapy |0|0.34246|0.237614|0.2345|0.298941|0.232403
C0012002_electrosurgery |0|0.322611|0.152729|0.167641|0.177737|0.146918
C00

C0338239_surgery, radioimmunoguided |0|0|0.249586|0.2254|0.270068|0.365622
C0339024_Intraocular tension relief (procedure) |0|0|0|0.24958|0.25659|0.359284
C0339597_glaucoma following surgery (diagnosis) |0|0|0|0|0|0
C0341090_Other post gastrointestinal tract surgery disorder NOS (disorder) |0|0|0|0|0|0
C0342398_Surgically-induced hypopituitarism |0|0|0|0|0|0
C0347890_Post gastrointestinal tract surgery hypoglycemia (disorder) |0|0|0|0|0|0
C0348002_ureteroileal conduit (treatment) |0|0.356796|0.174885|0.188524|0.214049|0.230259
C0348025_opening surgery |0|0.435341|0.275057|0.27186|0.30588|0.305437
C0348664_[X]Other functional disturbances following cardiac surgery (disorder) |0|0|0|0|0|0
C0348790_Vitreous syndrome following cataract surgery (disorder) |0|0|0|0|0|0
C0348829_insufficiency; pulmonary, following surgery, thoracic surgery, acute |0|0|0|0|0|0
C0348830_Acute pulmonary insufficiency following nonthoracic surgery (disorder) |0|0|0|0|0|0
C0348831_insufficiency; pulmonary, followi

C1719574_Bariatric surgery status complicating pregnancy, childbirth, or the puerperium, antepartum condition or complication |0|0|0|0|0|0
C1719575_Bariatric surgery status complicating pregnancy, childbirth, or the puerperium, postpartum condition or complication |0|0|0|0|0|0
C1719576_Bariatric surgery status complicating pregnancy, childbirth, or the puerperium |0|0|0|0|0|0
C1719579_Obesity surgery status complicating pregnancy, childbirth, or the puerperium |0|0|0|0|0|0
C1719686_bariatric surgery status (diagnosis) |0|0|0|0|0.24876|0.325582
C1719689_Obesity surgery status |0|0|0|0|0|0
C1719924_Timing of liver tumour staging pre-surgery |0|0|0|0|0|0
C1720208_Ophthalmic plastic surgery (qualifier value) |0|0.474865|0.162881|0.157389|0.200826|0.205947
C1720483_Vitreoretinal surgery specialty (qualifier value) |0|0|0|0|0|0
C1735501_Determine any contraindications to patient receiving oral medication (e.g., difficulty swallowing, nausea/vomiting, bowel inflammation, reduced peristalsis, 

C2458231_Stereotactic Particulate Radiosurgery of Sinuses |0|0|0|0|0|0
C2458232_Stereotactic Gamma Beam Radiosurgery of Sinuses |0|0|0|0|0|0
C2458233_Stereotactic Other Photon Radiosurgery of Hard Palate |0|0|0|0|0|0
C2458234_Stereotactic Particulate Radiosurgery of Hard Palate |0|0|0|0|0|0
C2458235_Stereotactic Gamma Beam Radiosurgery of Hard Palate |0|0|0|0|0|0
C2458236_Stereotactic Other Photon Radiosurgery of Soft Palate |0|0|0|0|0|0
C2458237_Stereotactic Particulate Radiosurgery of Soft Palate |0|0|0|0|0|0
C2458238_Stereotactic Gamma Beam Radiosurgery of Soft Palate |0|0|0|0|0|0
C2458239_Stereotactic Other Photon Radiosurgery of Larynx |0|0|0|0|0|0
C2458240_Stereotactic Particulate Radiosurgery of Larynx |0|0|0|0|0|0
C2458241_Stereotactic Gamma Beam Radiosurgery of Larynx |0|0|0|0|0|0
C2458242_Stereotactic Other Photon Radiosurgery of Pharynx |0|0|0|0|0|0
C2458243_Stereotactic Particulate Radiosurgery of Pharynx |0|0|0|0|0|0
C2458244_Stereotactic Gamma Beam Radiosurgery of Pharynx

C4067562_Patients who abstained from smoking prior to anesthesia on the day of surgery or procedure |0|0|0|0|0|0
C4067563_Patients who did not abstain from smoking prior to anesthesia on the day of surgery or procedure |0|0|0|0|0|0
C4068994_Ortho-Knee: Meniscal surgery (arthroscopic debridement, excision (menisectomy), or repair) |0|0|0|0|0|0
C4069014_Ortho-Elbow/wrist/hand: Carpal tunnel surgery |0|0|0|0|0|0
C4069272_Thoracic surgery involving thymectomy |0|0|0|0|0|0
C4069308_Oral and Maxillofacial Surgery Flowsheet |0|0|0|0|0|0
C4069310_Oral and Maxillofacial Surgery Education note |0|0|0|0|0|0
C4070855_Vascular surgery Preoperative evaluation and management note |0|0|0|0|0|0
C4070860_Transplant surgery Admit H&P note |0|0|0|0|0|0
C4070861_Preoperative evaluation and management note:Finding:Point in time:{Setting}:Document:Thoracic and cardiac surgery |0|0|0|0|0|0
C4070864_Plastic surgery Admit H&P note |0|0|0|0|0|0
C4070866_Surgical operation note:Finding:Point in time:{Setting}:Doc

...

C3880122_Flexible video reconstructive surgery endoscope (physical object) |0|0|0|0|0|0
p_value||0.0|0.0|0.0|0.0|0.0
C3880453_Open-surgery electrosurgical forceps, reusable (physical object) |0|0|0|0|0|0
p_value||0.0|0.0|0.0|0.0|0.0
C3880455_Cryoceutical treatment system pad, major surgery (physical object) |0|0|0|0|0|0
p_value||0.0|0.0|0.0|0.0|0.0
C3880650_Plastic surgery file (physical object) |0|0|0|0|0|0
p_value||0.0|0.0|0.0|0.0|0.0
C3880679_Rigid reconstructive surgery endoscope (physical object) |0|0|0|0|0|0
p_value||0.0|0.0|0.0|0.0|0.0
C3881184_Stereotactic surgery system workstation (physical object) |0|0|0|0|0|0
p_value||0.0|0.0|0.0|0.0|0.0
C3881481_Neurological stereotactic surgery system (physical object) |0|0|0|0|0|0
p_value||0.0|0.0|0.0|0.0|0.0
C3882020_Plastic surgery retractor, self-retaining (physical object) |0|0|0|0|0|0
p_value||0.0|0.0|0.0|0.0|0.0
C3882327_Fallopian tube dilator (physical object) |0|0|0|0|0|0
p_value||0.0|0.0|0.0|0.0|0.0
C3882335_Dental guided surger

C4062914_Previous breast surgery |0|0|0|0|0|0
p_value||0.0|0.0|0.0|0.0|0.0
C4065037_gastrointestinal surgery percutaneous sclerotherapy of fluid collection (treatment) |0|0|0|0|0|0
p_value||0.0|0.0|0.0|0.0|0.0
C4065872_comprehensive preoperative assessment for cataract surgery with lens placement (treatment) |0|0|0|0|0|0
p_value||0.0|0.0|0.0|0.0|0.0
C4067448_Received instruction from the anesthesiologist or proxy prior to the day of surgery to abstain from smoking on the day of surgery |0|0|0|0|0|0
p_value||0.0|0.0|0.0|0.0|0.0
C4067465_Req ret or w/in 90d of surg |0|0|0|0|0|0
p_value||0.0|0.0|0.0|0.0|0.0
C4067466_Patient did not require a return to the operating room within 90 days of surgery |0|0|0|0|0|0
p_value||0.0|0.0|0.0|0.0|0.0
C4067467_Patient achieved an improvement in visual acuity, from their preoperative level, within 90 days of surgery |0|0|0|0|0|0
p_value||0.0|0.0|0.0|0.0|0.0
C4067468_Patient did not achieve an improvement in visual acuity, from their preoperative level, w

C4071340_Flowsheet:Finding:Point in time:{Setting}:Document:Colon and rectal surgery |0|0|0|0|0|0
p_value||0.0|0.0|0.0|0.0|0.0
C4071421_Peds surgery Flowsheet |0|0|0|0|0|0
p_value||0.0|0.0|0.0|0.0|0.0
C4071422_Flowsheet:Finding:Point in time:{Setting}:Document:Thoracic and cardiac surgery |0|0|0|0|0|0
p_value||0.0|0.0|0.0|0.0|0.0
C4071428_Vascular surgery Flowsheet |0|0|0|0|0|0
p_value||0.0|0.0|0.0|0.0|0.0
C4071434_Surgical operation note:Finding:Point in time:{Setting}:Document:Neurological surgery |0|0|0|0|0|0
p_value||0.0|0.0|0.0|0.0|0.0
C4071456_Neurological surgery Flowsheet |0|0|0|0|0|0
p_value||0.0|0.0|0.0|0.0|0.0
C4071464_Neurological surgery Checklist |0|0|0|0|0|0
p_value||0.0|0.0|0.0|0.0|0.0
C4071486_Referral note:Finding:Point in time:{Setting}:Document:Colon and rectal surgery |0|0|0|0|0|0
p_value||0.0|0.0|0.0|0.0|0.0
C4075550_Microsurgery background material (physical object) |0|0|0|0|0|0
p_value||0.0|0.0|0.0|0.0|0.0
C4075673_Intraoperative hemorrhage requiring transfusion

C4285134_Postoperative evaluation and management note:Finding:Point in time:{Setting}:Document:Oral and maxillofacial surgery |0|0|0|0|0|0
p_value||0.0|0.0|0.0|0.0|0.0
C4285283_Plastic surgery Outpatient Note |0|0|0|0|0|0
p_value||0.0|0.0|0.0|0.0|0.0
C4285288_Vascular surgery Physician assistant Note |0|0|0|0|0|0
p_value||0.0|0.0|0.0|0.0|0.0
C4285293_Note:Finding:Point in time:{Setting}:Document:Neurological surgery.physician attending |0|0|0|0|0|0
p_value||0.0|0.0|0.0|0.0|0.0
C4285295_Vascular surgery.nurse practitioner |0|0|0|0|0|0
p_value||0.0|0.0|0.0|0.0|0.0
C4285302_Neurological surgery.physician resident |0|0|0|0|0|0
p_value||0.0|0.0|0.0|0.0|0.0
C4285306_Colon and rectal surgery.physician resident |0|0|0|0|0|0
p_value||0.0|0.0|0.0|0.0|0.0
C4285314_Orthopaedic surgery Hospital Note |0|0|0|0|0|0
p_value||0.0|0.0|0.0|0.0|0.0
C4285327_Vascular surgery Outpatient Postoperative evaluation and management note |0|0|0|0|0|0
p_value||0.0|0.0|0.0|0.0|0.0
C4285335_Plastic surgery Admit eval 

C4298112_Note:Finding:Point in time:{Setting}:Document:Colon and rectal surgery.nurse practitioner |0|0|0|0|0|0
p_value||0.0|0.0|0.0|0.0|0.0
C4298263_Note:Finding:Point in time:{Setting}:Document:Neurological surgery.nurse practitioner |0|0|0|0|0|0
p_value||0.0|0.0|0.0|0.0|0.0
C4298280_Views 1 or 2^during surgery:Finding:Point in time:Lower extremity>Knee:Document:XR |0|0|0|0|0|0
p_value||0.0|0.0|0.0|0.0|0.0
C4298407_Vascular surgery Initial evaluation note |0|0|0|0|0|0
p_value||0.0|0.0|0.0|0.0|0.0
C4298408_Peds surgery Initial eval note |0|0|0|0|0|0
p_value||0.0|0.0|0.0|0.0|0.0
C4298409_Transplant surgery Initial evaluation note |0|0|0|0|0|0
p_value||0.0|0.0|0.0|0.0|0.0
C4298410_Orthopaedic surgery Initial evaluation note |0|0|0|0|0|0
p_value||0.0|0.0|0.0|0.0|0.0
C4298411_Neurological surgery Initial evaluation note |0|0|0|0|0|0
p_value||0.0|0.0|0.0|0.0|0.0
C4298412_Initial evaluation note:Finding:Point in time:{Setting}:Document:Thoracic and cardiac surgery |0|0|0|0|0|0
p_value||0.0|

In [79]:
#chemotherapy
chemotherapy_list = []
with open('medication_dir/chemotherapy','r') as fout:
    for line in fout:
        chemotherapy_list.append(line.strip('\n'))

In [80]:
#white blood cancer 
#time_frame =['1809_1970','1971_1980','1981_1990','1991_2000','2001_2010','2011_2015'] #complete 'C0013217','C0392920','C3665472','C0282515','C0013216','C0184613',
for disease in ['C0023473']:
    print("{}_{}|{}".format(disease,concept_dict [disease][0] if disease in concept_dict.keys() else "","|".join(time_frame))) 
    for drug in chemotherapy_list+['C1268567','C1443775'] :
        get_score_sgns_pmi_disease_drug(drug,disease,model_dict_10,model_start="wi_",wc='10')

for disease in ['C0023473']: #,'C0023474','C1986083','C0349639'
    print("{}|{}".format(disease,"|".join(time_frame)))
    for drug in chemotherapy_list+['C1268567','C1443775'] :
        get_changpoint_score(model_dict_10,time_frame,disease,drug)

C0023473_myelosis; chronic |1809_1970|1971_1980|1981_1990|1991_2000|2001_2010|2011_2015
C0013217_pharmacotherapy |0.259611|0.372584|0.333473|0.346074|0.32555|0.323268
C0013218_polychemotherapy |0.38539|0.349377|0.323203|0.260431|0.225127|0.215219
C0018960_Photoradiation, Hematoporphyrin |0|0|0.229112|0.264078|0.211593|0.330282
C0031740_therapy, photodynamic |0|0.233823|0.158234|0.251681|0.247556|0.196057
C0034172_puva therapy |0|0.179724|0.236202|0.404259|0.264455|0.250141
C0050385_doxorubicin cyclophosphamide |0|0.382096|0.260335|0.265059|0.214204|0.194305
C0050856_doxorubicin, mitolactol, and mitomycins chemotherapy protocol |0|0|0|0|0|0
C0053334_bleomycin/cisplatin/etoposide |0|0.151811|0.101035|0.162421|0.203905|0.185234
C0053926_bleomycin/cisplatin/mitomycin/vincristine |0|0|0.258747|0.0822265|0.107099|0.160061
C0054427_cyclophosphamide/doxorubicin/fluorouracil protocol |0|0|0.282495|0.297671|0.264617|0.305705
C0054889_cyclophosphamide/doxorubicin/vincristine protocol |0|0|0.31036

C0034172_puva therapy |0|0.179724|0.236202|0.404259|0.264455|0.250141
p_value||0.1|0.03|0.02|0.27|0.38
C0050385_doxorubicin cyclophosphamide |0|0.382096|0.260335|0.265059|0.214204|0.194305
p_value||0.12|0.3|0.39|0.68|0.86
C0050856_doxorubicin, mitolactol, and mitomycins chemotherapy protocol |0|0|0|0|0|0
p_value||0.0|0.0|0.0|0.0|0.0
C0053334_bleomycin/cisplatin/etoposide |0|0.151811|0.101035|0.162421|0.203905|0.185234
p_value||0.18|0.14|0.09|0.04|0.3
C0053926_bleomycin/cisplatin/mitomycin/vincristine |0|0|0.258747|0.0822265|0.107099|0.160061
p_value||0.34|0.09|0.38|0.26|0.32
C0054427_cyclophosphamide/doxorubicin/fluorouracil protocol |0|0|0.282495|0.297671|0.264617|0.305705
p_value||0.35|0.03|0.07|0.3|0.19
C0054889_cyclophosphamide/doxorubicin/vincristine protocol |0|0|0.310365|0.310115|0.304283|0.265692
p_value||0.34|0.09|0.22|0.49|0.78
C0055598_cyclophosphamide/doxorubicin/prednisone/vincristine |0|0|0.279197|0.2472|0.158596|0.175587
p_value||0.34|0.1|0.22|0.39|0.41
C0056079_cyclopho

C0282515_prophylactic chemotherapy (treatment) |0.210422|0.324961|0.173611|0.178469|0.163216|0.1599
p_value||0.81|1.0|0.96|1.0|1.0
C0285131_cisplatin/methotrexate/vinblastine protocol |0|0|0.342222|0|0|0
p_value||0.82|0.46|1.0|1.0|1.0
C0286540_cytarabine/etoposide/mitoxantrone |0.181622|0.235385|0.173332|0.109855|0.128853|0.149756
p_value||0.79|1.0|1.0|0.86|0.7
C0341796_Post-chemotherapy testicular hypofunction (disorder) |0|0|0|0|0|0
p_value||0.0|0.0|0.0|0.0|0.0
C0342506_Postchemotherapy ovarian failure |0|0|0|0|0|0
p_value||0.0|0.0|0.0|0.0|0.0
C0362063_prophylactic; chemotherapy |0.0705403|0.115233|0.144413|0.082008|0.04282|0.0743101
p_value||0.3|0.55|0.94|0.93|0.6
C0371814_Insertion catheter artery |0|0|0|0|0|0
p_value||0.0|0.0|0.0|0.0|0.0
C0374469_intralesional chemotherapy administration up to and including 7 lesions (treatment) |0|0|0|0|0|0
p_value||0.0|0.0|0.0|0.0|0.0
C0374470_intralesional chemotherapy administration for more than 7 lesions (treatment) |0|0|0|0|0|0
p_value||0.0

C0597467_skin disorder pharmacotherapy |0|0|0|0|0|0
p_value||0.0|0.0|0.0|0.0|0.0
C0597625_urinary tract disorder pharmacotherapy |0|0|0|0|0|0
p_value||0.0|0.0|0.0|0.0|0.0
C0597869_AIDS chemotherapy |0|0|0.339405|0.275958|0.271826|0.230642
p_value||0.35|0.06|0.22|0.42|0.76
C0598325_drug addiction chemotherapy |0|0|0|0|0|0
p_value||0.0|0.0|0.0|0.0|0.0
C0659977_VIP protocol 3 |0|0|0|0|0|0
p_value||0.0|0.0|0.0|0.0|0.0
C0677881_cytotoxic chemotherapy |0.238293|0.274121|0.264965|0.223816|0.335348|0.276511
p_value||0.37|0.41|0.35|0.09|0.34
C0678054_electroporation therapy |0|0|0|0.158212|0.222854|0.153156
p_value||0.23|0.1|0.06|0.2|0.55
C0694988_Diphenhydramine hydrochloride, 50 mg, oral, fda approved prescription anti-emetic, for use as a complete therapeutic substitute for an iv anti-emetic at time of chemotherapy treatment not to exceed a 48 hour dosage regimen |0|0|0|0|0|0
p_value||0.0|0.0|0.0|0.0|0.0
C0694989_Prochlorperazine maleate, 5 mg, oral, fda approved prescription anti-emetic, fo

C1112312_Nausea post chemotherapy |0|0|0|0|0|0
p_value||0.0|0.0|0.0|0.0|0.0
C1112560_Diarrhoea post chemotherapy |0|0|0|0|0|0
p_value||0.0|0.0|0.0|0.0|0.0
C1135471_Brain wafer chemotherapy |0|0|0|0|0|0
p_value||0.0|0.0|0.0|0.0|0.0
C1135472_Interstitial/ intracavitary chemotherapy |0|0|0|0|0|0
p_value||0.0|0.0|0.0|0.0|0.0
C1144684_Implt brain chemotx add-on |0|0|0|0|0|0
p_value||0.0|0.0|0.0|0.0|0.0
C1161036_Chemotherapy care assessment (regime/therapy) |0|0|0|0|0|0
p_value||0.0|0.0|0.0|0.0|0.0
C1161037_chem edkshn |0|0|0|0.263089|0.161422|0
p_value||0.65|0.41|0.18|0.6|0.93
C1261324_[V]Chemotherapy follow-up (situation) |0|0|0|0|0|0
p_value||0.0|0.0|0.0|0.0|0.0
C1264521_effects of chemotherapy |0.303099|0.284826|0.258557|0.186956|0.282386|0.254804
p_value||1.0|1.0|0.94|0.57|0.73
C1272415_Manage chemotherapy care |0|0|0|0|0|0
p_value||0.0|0.0|0.0|0.0|0.0
C1273550_pre-operative chemotherapy administration (medication) |0|0|0.262004|0.242764|0.199563|0.164296
p_value||0.37|0.07|0.28|0.46|0.

C2045833_chemotherapy by intra-arterial infusion up to 1 hour (treatment) |0|0|0|0|0|0
p_value||0.0|0.0|0.0|0.0|0.0
C2045834_chemotherapy administration into peritoneal cavity (treatment) |0|0|0|0|0|0
p_value||0.0|0.0|0.0|0.0|0.0
C2045835_chemotherapy administration into pleural cavity (treatment) |0|0|0|0|0|0
p_value||0.0|0.0|0.0|0.0|0.0
C2045836_intralesional chemotherapy administration (treatment) |0|0|0|0|0|0
p_value||0.0|0.0|0.0|0.0|0.0
C2045837_chemotherapy arterial implantable pump filling or maintenance (treatment) |0|0|0|0|0|0
p_value||0.0|0.0|0.0|0.0|0.0
C2045838_chemotherapy by intrathecal injection (treatment) |0|0|0|0|0|0
p_value||0.0|0.0|0.0|0.0|0.0
C2045839_chemotherapy by intravenous infusion each additional sequential infusion (treatment) |0|0|0|0|0|0
p_value||0.0|0.0|0.0|0.0|0.0
C2045840_chemotherapy intravenous push technique |0|0|0|0|0|0
p_value||0.0|0.0|0.0|0.0|0.0
C2045841_chemotherapy by intravenous push technique each additional substance or drug (treatment) |0|

C2958031_rnox administer chemotherapy (treatment) |0|0|0|0|0|0
p_value||0.0|0.0|0.0|0.0|0.0
C2958043_rnox explain chemotherapy care (treatment) |0|0|0|0|0|0
p_value||0.0|0.0|0.0|0.0|0.0
C2958051_rnox coordinate chemotherapy care (treatment) |0|0|0|0|0|0
p_value||0.0|0.0|0.0|0.0|0.0
C2958052_rnox report chemotherapy complications (treatment) |0|0|0|0|0|0
p_value||0.0|0.0|0.0|0.0|0.0
C2966628_Flag for Date 1st chemotherapy treatment not populated:Prid:Pt:Cancer.XXX:Nom |0|0|0|0|0|0
p_value||0.0|0.0|0.0|0.0|0.0
C2986563_dose-dense chemotherapy |0|0|0|0.354689|0.292347|0.274123
p_value||0.58|0.21|0.03|0.19|0.55
C3161073_Antineoplastic chemotherapy induced pancytopenia |0|0|0|0|0|0
p_value||0.0|0.0|0.0|0.0|0.0
C3162051_rnox plan chemotherapy care (treatment) |0|0|0|0|0|0
p_value||0.0|0.0|0.0|0.0|0.0
C3166494_Type of chemotherapy |0.344187|0.43986|0.324467|0.315933|0.23547|0.195246
p_value||0.79|0.97|0.97|1.0|0.87
C3173311_Type of chemotherapy:Type:Pt:^Patient:Nom:PhenX |0|0|0|0|0|0
p_value|

C4038908_History of antineoplastic chemotherapy (situation) |0|0|0|0|0|0
p_value||0.0|0.0|0.0|0.0|0.0
C4051951_Culture of live tumor cells and chemotherapy drug response by staining |0|0|0|0|0|0
p_value||0.0|0.0|0.0|0.0|0.0
C4060804_Fragile vein (e.g., chemotherapy/radiation in limb to be injected, indwelling line for > 24 hours, axillary lymph node dissection in limb to be injected, distal intravenous access site) |0|0|0|0|0|0
p_value||0.0|0.0|0.0|0.0|0.0
C4087148_Immunochemotherapy |0|0|0|0|0|0
p_value||0.0|0.0|0.0|0.0|0.0
C4230496_Extreme sensitivity to chemotherapy |0|0|0|0|0|0
p_value||0.0|0.0|0.0|0.0|0.0
C4269122_Newborn affected by cancer chemotherapy |0|0|0|0|0|0
p_value||0.0|0.0|0.0|0.0|0.0
C4272534_palliative chemotherapy for malignant neoplasm (treatment) |0|0|0|0|0|0
p_value||0.0|0.0|0.0|0.0|0.0
C4272610_neoadjuvant chemotherapy for malignant neoplasm (treatment) |0|0|0|0|0|0
p_value||0.0|0.0|0.0|0.0|0.0
C4272699_maintenance chemotherapy for malignant neoplasm (treatment) |

In [87]:
#chemotherapy
daa_list = []
with open('medication_dir/DAA','r') as fout:
    for line in fout:
        daa_list.append(line.strip('\n'))

In [88]:
# happatitis C
hc_disease = ["C0220847"] #"C0019196",
hc_drug = ["C0035525","C0073220","C0073221","C0073222","C0391001","C0796545","C3653501","C3540755","C3541376",\
           "C3541969","C3653392","C3653443","C3653892",
           "C1566826","C2605855","C3852670",\
           "C1675326","C1738934","C1876229","C3852512"]+daa_list

#time_frame = ['1809_1970','1971_1980','1981_1990','1991_2000','2001_2010','2011_2015'] #complete
for disease in hc_disease:
    
    print("{}_{}|{}".format(disease,concept_dict [disease][0] if disease in concept_dict.keys() else "","|".join(time_frame))) 
    for drug in hc_drug:
        get_score_sgns_pmi_disease_drug(drug,disease,model_dict_10,model_start="wi_",wc='10')

for disease in hc_disease:
    print("{}|{}".format(disease,"|".join(time_frame)))
    for drug in hc_drug:
        get_changpoint_score(model_dict_10,time_frame,disease,drug)
        #print("\n")

C0220847_post-transfusion hepatitis non A non B virus |1809_1970|1971_1980|1981_1990|1991_2000|2001_2010|2011_2015
C0035525_ribovirin |0|0.260635|0.275498|0.577654|0.571129|0.592925
C0073220_ribavirin-TP |0|0.373291|0.277672|0.445364|0.381784|0
C0073221_viramidine |0|0|0.191573|0.429416|0.361891|0.401547
C0073222_ribavirin-5'-phosphate |0|0.341761|0|0|0|0
C0391001_polyethylene glycol-interferon alpha-2a |0|0|0|0.45083|0.490017|0.52137
C0796545_polyethylene glycol-interferon alpha-2b |0|0|0|0.413528|0.434866|0.445201
C3653501_DIRECT ACTING ANTIVIRALS |0|0|0|0|0.555836|0.653779
C3540755_Protease inhibitors, direct acting antivirals |0|0.18055|0.119078|0.241043|0.368348|0.44366
C3541376_Non-nucleoside reverse transcriptase inhibitors |0|0|0|0.293492|0.435286|0.431129
C3541969_Neuraminidase inhibitors, direct acting antivirals |0|0.352425|0|0.263841|0.29057|0.272151
C3653392_Phosphonic acid derivatives, direct acting antivirals |0|0.238207|0.191442|0.103184|0.122256|0.237834
C3653443_Cycli

In [86]:
#CC

for disease in ['C0021344']:
    
    #['C0153446','C0279637',"C0007847","C0302592","C4048328",'C0021344','C0343641',\
                 #'C0375071','C0677055','C0042237','C0262659','C0238519','C0238518']:
    
    print("{}_{}|{}".format(disease,concept_dict [disease][0] if disease in concept_dict.keys() else "","|".join(time_frame))) 
    for drug in chemotherapy_list+['C3665472','C1522449','C1721787','C1512511','C0390856'] :
         get_score_sgns_pmi_disease_drug(drug,disease,model_dict_10,model_start="wi_",wc='10')
            
for disease in ['C0021344']:
    print("{}|{}".format(disease,"|".join(time_frame)))
    for drug in chemotherapy_list+['C3665472','C1522449','C1721787','C1512511','C0390856']:
        get_changpoint_score(model_dict_10,time_frame,disease,drug)

C0021344_warts virus |1809_1970|1971_1980|1981_1990|1991_2000|2001_2010|2011_2015
C0013217_pharmacotherapy |0.266683|0.13597|0.182149|0.182505|0.184491|0.267664
C0013218_polychemotherapy |0.289002|0.179214|0.1839|0.192204|0.174753|0.14818
C0018960_Photoradiation, Hematoporphyrin |0|0|0.283249|0.283754|0.253091|0.266946
C0031740_therapy, photodynamic |0|0.266714|0.281367|0.210014|0.26241|0.258596
C0034172_puva therapy |0|0.29482|0.23884|0.275189|0.301966|0.267876
C0050385_doxorubicin cyclophosphamide |0|0.167348|0.149995|0.180073|0.172995|0.149207
C0050856_doxorubicin, mitolactol, and mitomycins chemotherapy protocol |0|0|0|0|0|0
C0053334_bleomycin/cisplatin/etoposide |0|0.173393|0.0403137|0.151893|0.145757|0.100588
C0053926_bleomycin/cisplatin/mitomycin/vincristine |0|0|0.145843|0.181126|0.0827819|0.18959
C0054427_cyclophosphamide/doxorubicin/fluorouracil protocol |0|0|0.208774|0.170229|0.174853|0.24156
C0054889_cyclophosphamide/doxorubicin/vincristine protocol |0|0|0.125545|0.174399|0

C0013217_pharmacotherapy |0.266683|0.13597|0.182149|0.182505|0.184491|0.267664
p_value||0.82|0.32|0.22|0.11|0.17
C0013218_polychemotherapy |0.289002|0.179214|0.1839|0.192204|0.174753|0.14818
p_value||1.0|0.86|0.89|1.0|1.0
C0018960_Photoradiation, Hematoporphyrin |0|0|0.283249|0.283754|0.253091|0.266946
p_value||0.3|0.09|0.16|0.4|0.5
C0031740_therapy, photodynamic |0|0.266714|0.281367|0.210014|0.26241|0.258596
p_value||0.11|0.21|0.47|0.38|0.67
C0034172_puva therapy |0|0.29482|0.23884|0.275189|0.301966|0.267876
p_value||0.11|0.21|0.16|0.21|0.73
C0050385_doxorubicin cyclophosphamide |0|0.167348|0.149995|0.180073|0.172995|0.149207
p_value||0.19|0.17|0.1|0.44|0.8
C0050856_doxorubicin, mitolactol, and mitomycins chemotherapy protocol |0|0|0|0|0|0
p_value||0.0|0.0|0.0|0.0|0.0
C0053334_bleomycin/cisplatin/etoposide |0|0.173393|0.0403137|0.151893|0.145757|0.100588
p_value||0.16|0.46|0.26|0.52|0.73
C0053926_bleomycin/cisplatin/mitomycin/vincristine |0|0|0.145843|0.181126|0.0827819|0.18959
p_valu

C0281548_intraperitoneal therapy |0.402211|0.292209|0.156472|0.130823|0.194417|0.160656
p_value||1.0|1.0|0.87|0.77|0.7
C0282515_prophylactic chemotherapy (treatment) |0.414104|0.146061|0.187325|0.195885|0.183111|0.200096
p_value||1.0|0.7|0.55|0.55|0.31
C0285131_cisplatin/methotrexate/vinblastine protocol |0|0|0.274035|0|0|0
p_value||0.86|0.64|1.0|1.0|1.0
C0286540_cytarabine/etoposide/mitoxantrone |0.46058|0.221075|0.128395|0.0797633|0.0846799|0.0298724
p_value||0.96|1.0|1.0|0.95|1.0
C0341796_Post-chemotherapy testicular hypofunction (disorder) |0|0|0|0|0|0
p_value||0.0|0.0|0.0|0.0|0.0
C0342506_Postchemotherapy ovarian failure |0|0|0|0|0|0
p_value||0.0|0.0|0.0|0.0|0.0
C0362063_prophylactic; chemotherapy |0.33176|0.124465|0.155341|0.174902|0.116227|0.195136
p_value||1.0|0.84|0.73|0.74|0.36
C0371814_Insertion catheter artery |0|0|0|0|0|0
p_value||0.0|0.0|0.0|0.0|0.0
C0374469_intralesional chemotherapy administration up to and including 7 lesions (treatment) |0|0|0|0|0|0
p_value||0.0|0.0|0

C0598325_drug addiction chemotherapy |0|0|0|0|0|0
p_value||0.0|0.0|0.0|0.0|0.0
C0659977_VIP protocol 3 |0|0|0|0|0|0
p_value||0.0|0.0|0.0|0.0|0.0
C0677881_cytotoxic chemotherapy |0.347755|0.161098|0.164254|0.158842|0.147622|0.168728
p_value||1.0|0.91|0.83|0.82|0.24
C0678054_electroporation therapy |0|0|0|0.214926|0.248764|0.233657
p_value||0.47|0.25|0.07|0.08|0.26
C0694988_Diphenhydramine hydrochloride, 50 mg, oral, fda approved prescription anti-emetic, for use as a complete therapeutic substitute for an iv anti-emetic at time of chemotherapy treatment not to exceed a 48 hour dosage regimen |0|0|0|0|0|0
p_value||0.0|0.0|0.0|0.0|0.0
C0694989_Prochlorperazine maleate, 5 mg, oral, fda approved prescription anti-emetic, for use as a complete therapeutic substitute for an iv anti-emetic at the time of chemotherapy treatment, not to exceed a 48 hour dosage regimen |0|0|0|0|0|0
p_value||0.0|0.0|0.0|0.0|0.0
C0694990_Prochlorperazine maleate10mg |0|0|0|0|0|0
p_value||0.0|0.0|0.0|0.0|0.0
C069499

C1112312_Nausea post chemotherapy |0|0|0|0|0|0
p_value||0.0|0.0|0.0|0.0|0.0
C1112560_Diarrhoea post chemotherapy |0|0|0|0|0|0
p_value||0.0|0.0|0.0|0.0|0.0
C1135471_Brain wafer chemotherapy |0|0|0|0|0|0
p_value||0.0|0.0|0.0|0.0|0.0
C1135472_Interstitial/ intracavitary chemotherapy |0|0|0|0|0|0
p_value||0.0|0.0|0.0|0.0|0.0
C1144684_Implt brain chemotx add-on |0|0|0|0|0|0
p_value||0.0|0.0|0.0|0.0|0.0
C1161036_Chemotherapy care assessment (regime/therapy) |0|0|0|0|0|0
p_value||0.0|0.0|0.0|0.0|0.0
C1161037_chem edkshn |0|0|0|0.160927|0.18909|0
p_value||0.65|0.39|0.22|0.34|1.0
C1261324_[V]Chemotherapy follow-up (situation) |0|0|0|0|0|0
p_value||0.0|0.0|0.0|0.0|0.0
C1264521_effects of chemotherapy |0.303276|0.153236|0.119333|0.133937|0.166093|0.223233
p_value||1.0|0.92|0.62|0.38|0.18
C1272415_Manage chemotherapy care |0|0|0|0|0|0
p_value||0.0|0.0|0.0|0.0|0.0
C1273550_pre-operative chemotherapy administration (medication) |0|0|0.236203|0.15775|0.235204|0.18261
p_value||0.33|0.05|0.19|0.14|0.53

C2045840_chemotherapy intravenous push technique |0|0|0|0|0|0
p_value||0.0|0.0|0.0|0.0|0.0
C2045841_chemotherapy by intravenous push technique each additional substance or drug (treatment) |0|0|0|0|0|0
p_value||0.0|0.0|0.0|0.0|0.0
C2045842_chemotherapy by arterial injection (treatment) |0|0|0|0|0|0
p_value||0.0|0.0|0.0|0.0|0.0
C2045843_chemotherapy by intracavitary injection (treatment) |0|0|0|0|0|0
p_value||0.0|0.0|0.0|0.0|0.0
C2045844_chemotherapy by intrathecal injection via reservoir (treatment) |0|0|0|0|0|0
p_value||0.0|0.0|0.0|0.0|0.0
C2045845_intraventricular chemotherapy by injection via reservoir (treatment) |0|0|0|0|0|0
p_value||0.0|0.0|0.0|0.0|0.0
C2045846_intraventricular chemotherapy by injection via reservoir of multiple agents (treatment) |0|0|0|0|0|0
p_value||0.0|0.0|0.0|0.0|0.0
C2045847_intraventricular chemotherapy by injection via reservoir of single agent (treatment) |0|0|0|0|0|0
p_value||0.0|0.0|0.0|0.0|0.0
C2045848_chemotherapy by subarachnoid injection (treatment

C3166494_Type of chemotherapy |0.388407|0.193376|0.242612|0.150076|0.19864|0.261116
p_value||1.0|0.83|0.85|0.46|0.34
C3173311_Type of chemotherapy:Type:Pt:^Patient:Nom:PhenX |0|0|0|0|0|0
p_value||0.0|0.0|0.0|0.0|0.0
C3176017_Have you received cancer chemotherapy in the past 4W or do you anticipate such therapy in the next 4W:Finding:Point in time:^Patient:Ordinal:PhenX |0|0|0|0|0|0
p_value||0.0|0.0|0.0|0.0|0.0
C3176018_Have you received cancer chemotherapy in the past 4W or do you anticipate such therapy in the next 4W |0|0|0|0|0|0
p_value||0.0|0.0|0.0|0.0|0.0
C3178761_Radiochemotherapy, Adjuvant |0|0|0.299905|0.292357|0.26936|0.347456
p_value||0.22|0.07|0.11|0.2|0.23
C3178775_Synchronous Chemoradiotherapy |0|0|0.264894|0.305528|0.326131|0.389908
p_value||0.38|0.06|0.04|0.09|0.19
C3179010_Induction chemotherapy (procedure) |0|0.149755|0.186011|0.211916|0.204475|0.312562
p_value||0.17|0.03|0.04|0.14|0.2
C3203481_Sterility post chemotherapy |0|0|0|0|0|0
p_value||0.0|0.0|0.0|0.0|0.0
C3248

C4087148_Immunochemotherapy |0|0|0|0|0|0
p_value||0.0|0.0|0.0|0.0|0.0
C4230496_Extreme sensitivity to chemotherapy |0|0|0|0|0|0
p_value||0.0|0.0|0.0|0.0|0.0
C4269122_Newborn affected by cancer chemotherapy |0|0|0|0|0|0
p_value||0.0|0.0|0.0|0.0|0.0
C4272534_palliative chemotherapy for malignant neoplasm (treatment) |0|0|0|0|0|0
p_value||0.0|0.0|0.0|0.0|0.0
C4272610_neoadjuvant chemotherapy for malignant neoplasm (treatment) |0|0|0|0|0|0
p_value||0.0|0.0|0.0|0.0|0.0
C4272699_maintenance chemotherapy for malignant neoplasm (treatment) |0|0|0|0|0|0
p_value||0.0|0.0|0.0|0.0|0.0
C4272720_leukemia due to chemotherapy (diagnosis) |0|0|0|0|0|0
p_value||0.0|0.0|0.0|0.0|0.0
C4272769_interim maintenance chemotherapy for malignant neoplasm (treatment) |0|0|0|0|0|0
p_value||0.0|0.0|0.0|0.0|0.0
C4272796_induction chemotherapy for malignant neoplasm (treatment) |0|0|0|0|0|0
p_value||0.0|0.0|0.0|0.0|0.0
C4272989_delayed intensification chemotherapy for malignant neoplasm (treatment) |0|0|0|0|0|0
p_valu

In [90]:
## ulcer surgery_list+
drug_list = ['C0038895','C0543467','C1457907',"C0038894","C1274039",'C1547138','C1705196','C4084778',"C0017118","C0399839","C0161899","C0581065","C0023038" ]
for disease in ['C0030920']:
    
    #['C0153446','C0279637',"C0007847","C0302592","C4048328",'C0021344','C0343641',\
                 #'C0375071','C0677055','C0042237','C0262659','C0238519','C0238518']:
    
    print("{}_{}|{}".format(disease,concept_dict [disease][0] if disease in concept_dict.keys() else "","|".join(time_frame))) 
    for drug in drug_list :
         get_score_sgns_pmi_disease_drug(drug,disease,model_dict_10,model_start="wi_",wc='10')
            
for disease in ['C0030920']:
    print("{}|{}".format(disease,"|".join(time_frame)))
    for drug in drug_list:
        get_changpoint_score(model_dict_10,time_frame,disease,drug)

C0030920_ulcer; peptic |1809_1970|1971_1980|1981_1990|1991_2000|2001_2010|2011_2015
C0038895_surgery |0.303991|0.308407|0.327271|0.274342|0.282563|0.250761
C0543467_surgical treatments |0.271196|0.306109|0.316994|0.29852|0.256257|0.220214
C1457907_surgery |0.316389|0.23398|0.171225|0.242545|0.194063|0.180778
C0038894_Surgery specialty |0.354777|0.257236|0.299403|0.234917|0.30391|0.239166
C1274039_surgery general |0.29694|0.237063|0.341872|0.290218|0.299904|0.251361
C1547138_Surgery |0|0|0|0|0|0
C1705196_Surgery Domain |0|0|0|0.295107|0.181454|0.172808
C4084778_Surgery - Clinical Class |0|0|0|0|0|0
C0017118_gastrectomy (treatment) |0.577877|0.611078|0.548266|0.511217|0.4547|0.41574
C0399839_gastrojejunostomy (treatment) |0.57022|0.534482|0.510732|0.43408|0.425911|0.37167
C0161899_total gastrectomy (treatment) |0.511046|0.519105|0.460631|0.416665|0.363231|0.381484
C0581065_Triple therapy helicobacter pylori (procedure) |0|0|0|0.568703|0.610672|0.509389
C0023038_laparotomy procedure |0.28

In [529]:
def get_stats_proto(ts,concept='C0344452'):
    normalised_score= {}
    for time in time_frame:
        k_sgns='wi_'+time+'_fastext_10_300_5.bin' 
        model=model_dict[k_sgns]
        #list_neighbours = get_neighbours(concept,model)
        #mean,variance,std,scores = get_mean_similarity_variance_at_time_frame(concept,model,list_neighbours)
        #print("time: {}, mean: {}, variance: {}, deviation {}, nums {}".format(time,mean,variance,std,\
                                                                                         #len(list_neighbours)))
    if bool(variance) and bool(mean):
        for score in ts :
            n_score = (score-mean)/np.sqrt(variance)
            p_value = scipy.stats.norm.sf(abs(n_score))
            normalised_score[score] = [n_score,p_value]
        #else:
            #normalised_score.append(0)
    #print(normalised_score)        
    #model_change = MeanShiftModel()
    #stats_ts, pvals, nums = model_change.detect_mean_shift(normalised_score, B=1000)
    #print("stats_ts: {}, pvals: {}, nums: {}".format(stats_ts, pvals, nums))
    #print(" pvals: {}".format(pvals))
    #return normalised_score

In [738]:
def get_mean_similarity_variance_at_time_frame(concept,model,list_neighbours):
    key_set = model.wv.vocab.keys()
    if concept in key_set:
        scores = [model.wv.similarity(concept,key)if key in key_set  else 0 for key in list_neighbours]
        _mean = np.mean(scores)
        _var = np.var(scores)
        _sd = np.std(scores)
        return _mean,_var,_sd
    else:
        return None,None,None
    
def get_neighbours_drug(concept,model,topN=100):
    drugs_list = ['Clinical Drug','Antibiotic','Diagnostic Procedure','Drug Delivery Device','Gene or Genome','Pharmacologic Substance','Vitamin''Antibiotic','Diagnostic Procedure','Clinical Drug','Drug Delivery Device','Gene or Genome','Medical Device',\
'Medical Device','Nucleic Acid, Nucleoside, or Nucleotide', 'Therapeutic or Preventive Procedure','Vitamin','Organic Chemical',"Amino Acid, Peptide, or Protein"]
    neighbour_list = []
    key_set = list(model.wv.vocab.keys())
    for neighbor in key_set :
            if neighbor in concept_dict.keys()  :
                get = False
                for word in drugs_list:
                    if word in concept_dict[neighbor][1]:
                        get = True
                    if get:
                        neighbour_list.append(neighbor)
                        break
            if len(neighbour_list)>=topN:
                #print(neighbour_list)
                break
                
    return neighbour_list
        
def get_neighbours(concept,model,topN=100):
    key_set = model.wv.vocab.keys()
    random_neighbours = np.random.choice(list(key_set), size=topN, replace=False)
    return random_neighbours #list(model.wv.vocab.keys())

In [739]:
def get_stats(score,model,disease,time,drug,random=True):
    normalised_score= []
    for _ in range(100):
        if random:
            list_neighbours = get_neighbours(disease,model)
        else:
            list_neighbours = get_neighbours_drug(disease,model)

        mean,variance,std = get_mean_similarity_variance_at_time_frame(disease,model,list_neighbours)
        #print("time: {}, mean: {}, variance: {}, deviation {}, nums {}".format(time,mean,variance,std,\
                                                                                             #len(list_neighbours)))
        if bool(variance) and bool(mean):
            n_score = (score-mean)/np.sqrt(variance)
            p_value = scipy.stats.norm.sf(abs(n_score))
            normalised_score.append(p_value) #[n_score,p_value]
            #print(normalised_score)
            
    return np.mean(normalised_score)


In [740]:
def get_significance_score_disease_drug_at_time_frame(disease,drug_list,random):
    print('{}| {}'.format(disease,'|'.join(time_frame)))
    for drug in drug_list:
        normalised_score = {}
        for time in time_frame: #time_frame
            k_sgns='wi_'+time+'_fastext_5_300_5.bin'
            model = model_dict[k_sgns]
            if disease in model.wv.vocab.keys() and drug in  model.wv.vocab.keys() :
                score= model.wv.similarity(disease,drug)
                normalised_score[time] = get_stats(score,model,disease,time,drug, random)
            else:
                normalised_score[time] = 'NAN'
            #print(time,normalised_score[time] )
                
        print('{}|{}'.format(drug,'|'.join(map(str,[normalised_score[time] for time in time_frame]) )))

In [751]:
# top K neighbour apporach
def top_K_neighbours_similarity_with_disease_drug_neighbour(disease_list,drug_list,topN=10):
    for disease in disease_list:
        print('{}| {}'.format(disease,'|'.join(time_frame)))
        for drug in drug_list:
            avg_drug_score = {}
            for time in time_frame:
                k_sgns='wi_'+time+'_fastext_5_300_5.bin'
                model = model_dict[k_sgns]
                if disease in model.wv.vocab.keys() and drug in  model.wv.vocab.keys() :
                    disease_neighbour_list = [neighbour[0] for neighbour in model.wv.most_similar(disease,topn=topN)]
                    drug_neighbour_list = [neighbour[0] for neighbour in model.wv.most_similar(drug,topn=topN)]
                    score_list = []
                    for disease_neighbour in disease_neighbour_list:
                        for drug_neighbour in drug_neighbour_list:
                            score_list.append(model.wv.similarity(disease_neighbour,drug_neighbour))
                    avg_drug_score[time] = np.mean(score_list)
                else:
                    
                    avg_drug_score[time] = 'NAN'
            print('{}| {}'.format(drug,'|'.join(map(str,[avg_drug_score[time] for time in time_frame]) )))
                            
                
    
    
    

In [752]:
# prolactinoma
disease_list = ["C0344452"] 
drug_list = ["C0107994","C2985562",'C0256089','C0006230','C0546852' ]#  'C2985562','C0546852'
#print('random 100 neighbours')
#get_significance_score_disease_drug_at_time_frame(disease_list[0],drug_list,random=True)
#print('random 100 drug neighbours')
#get_significance_score_disease_drug_at_time_frame(disease_list[0],drug_list,random=False)
#get_change_point(disease_list[0],drug_list)
print('average similarity between ALL top 10 neighbours combination of drug and diease')
top_K_neighbours_similarity_with_disease_drug_neighbour(disease_list,drug_list)

average similarity between ALL top k neighbours combination of drug and diease
C0344452| 1809_1970|1971_1980|1981_1990|1991_2000|2001_2010|2011_2015
C0107994| NAN|NAN|0.518066|0.471986|0.415222|0.501215
C2985562| NAN|0.537847|0.584504|0.571396|0.50116|0.473903
C0256089| NAN|NAN|NAN|0.526746|0.45568|0.434474
C0006230| NAN|0.39396|0.50013|0.507164|0.435|0.417878
C0546852| NAN|0.501655|0.522642|0.393834|0.351992|0.354604


In [753]:
#hc
list_disease = ["C0220847"]
list_drug = ["C0035525","C3653501","C0391001","C0796545"]

#print('random 100 neighbours')
#get_significance_score_disease_drug_at_time_frame(list_disease[0],list_drug,random=True)
#print('random 100 drug neighbours')
#get_significance_score_disease_drug_at_time_frame(list_disease[0],list_drug,random=False)
#print('changepoint')
#get_change_point(list_disease[0],list_drug) 

print('average similarity between ALL top 10 neighbours combination of drug and diease')
top_K_neighbours_similarity_with_disease_drug_neighbour(list_disease,list_drug)

average similarity between ALL top k neighbours combination of drug and diease
C0220847| 1809_1970|1971_1980|1981_1990|1991_2000|2001_2010|2011_2015
C0035525| NAN|0.404286|0.264413|0.584076|0.481706|0.534199
C3653501| NAN|NAN|NAN|NAN|0.438428|0.524432
C0391001| NAN|NAN|NAN|0.319804|0.475065|0.538767
C0796545| NAN|NAN|NAN|0.33792|0.478959|0.539033


In [754]:
# white blood cell cancer
list_disease=['C0023473','C0023474','C0349639']
list_drug=['C3665472','C0282515','C0013216','C0184613','C1268567','C1443775']
"""for disease in list_disease:

    print('random 100 neighbours')
    get_significance_score_disease_drug_at_time_frame(disease,list_drug,random=True)
    print('random 100 drug neighbours')
    get_significance_score_disease_drug_at_time_frame(disease,list_drug,random=False)
    print('changepoint')
    get_change_point(disease,list_drug)"""
print('average similarity between ALL top 10 neighbours combination of drug and diease')
top_K_neighbours_similarity_with_disease_drug_neighbour(list_disease,list_drug)

average similarity between ALL top 10 neighbours combination of drug and diease
C0023473| 1809_1970|1971_1980|1981_1990|1991_2000|2001_2010|2011_2015
C3665472| 0.386241|0.32529|0.30065|0.311912|0.316849|0.3165
C0282515| 0.424355|0.325072|0.324957|0.269189|0.252186|0.231233
C0013216| 0.329024|0.254823|0.187334|0.192207|0.236433|0.261811
C0184613| NAN|0.23584|0.215417|0.237986|0.217118|0.238284
C1268567| NAN|NAN|0.252875|0.195242|0.463625|0.553112
C1443775| NAN|NAN|0.252859|0.215573|0.394485|0.412137
C0023474| 1809_1970|1971_1980|1981_1990|1991_2000|2001_2010|2011_2015
C3665472| 0.473115|0.404698|0.333209|0.313203|0.319622|0.332327
C0282515| 0.596444|0.417586|0.360327|0.272076|0.256643|0.228749
C0013216| 0.389829|0.34547|0.214848|0.197329|0.246398|0.275348
C0184613| NAN|0.318147|0.235141|0.244033|0.226429|0.247879
C1268567| NAN|NAN|0.258597|0.19883|0.468178|0.569196
C1443775| NAN|NAN|0.26099|0.218315|0.400978|0.425177
C0349639| 1809_1970|1971_1980|1981_1990|1991_2000|2001_2010|2011_2015


In [756]:
#cervical cancer
list_disease=['C0153446','C0021344'] #,'C0279637','C0021344','C0375071','C0677055','C0262659'
list_drug =['C3665472','C1522449','C1721787','C1512511','C0390856'] 
"""for disease in list_disease:

    print('random 100 neighbours')
    get_significance_score_disease_drug_at_time_frame(disease,list_drug,random=True)
    print('random 100 drug neighbours')
    get_significance_score_disease_drug_at_time_frame(disease,list_drug,random=False)
    print('changepoint')
    get_change_point(disease,list_drug)"""

print('average similarity between ALL top 10 neighbours combination of drug and diease')
top_K_neighbours_similarity_with_disease_drug_neighbour(list_disease,list_drug)

average similarity between ALL top 10 neighbours combination of drug and diease
C0153446| 1809_1970|1971_1980|1981_1990|1991_2000|2001_2010|2011_2015
C3665472| NAN|NAN|0.401143|0.370803|0.372146|0.339643
C1522449| NAN|NAN|0.296793|0.307788|0.311655|0.291431
C1721787| NAN|NAN|NAN|NAN|0.349398|0.405155
C1512511| NAN|NAN|0.335609|0.379693|0.338332|0.363162
C0390856| NAN|NAN|NAN|0.326195|0.379118|NAN
C0021344| 1809_1970|1971_1980|1981_1990|1991_2000|2001_2010|2011_2015
C3665472| 0.353474|0.29135|0.172268|0.183656|0.189944|0.220602
C1522449| 0.27509|0.229445|0.159517|0.175206|0.17609|0.164438
C1721787| NAN|NAN|NAN|NAN|0.475723|0.493466
C1512511| 0.63779|0.51633|0.309464|0.377464|0.457815|0.496114
C0390856| NAN|NAN|NAN|0.268761|0.445146|NAN


In [747]:
# stomach cancer/ ulcer
list_disease=["C0038358","C0030920","C0013295","C0030922"]
list_drug = ["C0017118","C0399839","C0161899","C0581065","C0023038", "C0192353" ]
for disease in list_disease:

    print('random 100 neighbours')
    get_significance_score_disease_drug_at_time_frame(disease,list_drug,random=True)
    print('random 100 drug neighbours')
    get_significance_score_disease_drug_at_time_frame(disease,list_drug,random=False)
    print('changepoint')
    get_change_point(disease,list_drug)


random 100 neighbours
C0038358| 1809_1970|1971_1980|1981_1990|1991_2000|2001_2010|2011_2015
C0017118|0.000962232832601|1.68399124261e-06|1.58729538988e-06|2.91464516016e-06|4.09620138929e-05|0.000442547166141
C0399839|0.000168360579355|1.90724991156e-05|8.0623315626e-06|6.10581017603e-05|1.21793339631e-05|0.000924755629192
C0161899|0.0116169663312|1.62445301987e-05|2.94195407547e-05|0.0001063844704|0.00066234635008|0.00297590405732
C0581065|NAN|NAN|NAN|2.80878799469e-06|7.46710497901e-05|0.00347165482012
C0023038|0.272648925052|0.0102318499263|0.012361824476|0.00389841458883|0.00600890429279|0.0271008551547
C0192353|0.000651488924688|NAN|0.000186723811384|0.00272887359451|0.00755573915554|0.0274960067663
random 100 drug neighbours
C0038358| 1809_1970|1971_1980|1981_1990|1991_2000|2001_2010|2011_2015
C0017118|0.000400245924189|1.95968046953e-08|1.35093978493e-06|9.68532400329e-08|2.78872135284e-05|0.000108932153017
C0399839|4.98773690788e-05|1.1271695433e-06|3.67059294725e-06|5.97684299

C0192353 | 1809_1970|1971_1980|1981_1990|1991_2000|2001_2010|2011_2015
C0192353|0.824|0.19|0.432|0.534|0.827


In [23]:
def get_change_point(disease,drug_list):
    drug_pvalue_dict = []
    for drug in drug_list:
        normalised_drug_score_across_time = []
        for time in time_frame:
            k_sgns='wi_'+time+'_fastext_5_300_5.bin'
            model = model_dict[k_sgns]
            if disease in model.wv.vocab.keys():
                list_neighbours = get_neighbours_drug(disease,model)
                mean,variance,std = get_mean_similarity_variance_at_time_frame(disease,model,list_neighbours)
                if drug in model.wv.vocab.keys():
                    simi_score = model.wv.similarity(disease,drug)
                else:
                    simi_score = 0
                normalised_drug_score_across_time.append((simi_score-mean)/(std))
            else:
                normalised_drug_score_across_time.append(0)
        #print(len(normalised_drug_score_across_time))
        #print(normalised_drug_score_across_time)
            
        model_change = MeanShiftModel()
        stats_ts, pvals, nums = model_change.detect_mean_shift(normalised_drug_score_across_time, B=1000)
        #print(stats_ts, pvals, nums)
        #drug_pvalue_dict[drug] = pvals
        print('{} | {}'.format(drug,'|'.join(time_frame)))
        print('{}||{}'.format(drug,'|'.join(map(str,pvals))))
    

In [689]:
def compute_balance_mean_ts(ts):
    balance = [compute_balance_mean(ts, t) for t in np.arange(0, len(ts)-1)]
    #print(len(balance))
    return balance
def compute_balance_mean( ts, t):
    return np.mean(ts[t + 1:]) - np.mean(ts[:t + 1])

def shuffle_timeseries(ts):
    return np.random.permutation(ts)

#stat_ts_func = self.compute_balance_mean_ts
#null_ts_func = self.shuffle_timeseries

In [691]:
stats_ts = compute_balance_mean_ts(ts)
null_ts = map(np.random.permutation, np.array([ts, ] * 10))
stats_null_ts = np.vstack([compute_balance_mean_ts(nts) for nts in null_ts])
pvals = []
nums = []
for i in np.arange(0, len(stats_ts)):
    num_samples = np.sum((stats_null_ts[:, i] >= stats_ts[i]))
    nums.append(num_samples)
    pval = num_samples / float(10)
    pvals.append(pval)
print(pvals)


[0.40000000000000002, 0.59999999999999998, 0.90000000000000002, 0.90000000000000002]
